# Perceptron

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

## Uczenie perceptronu

Uczenie perceptronu to iteracyjny algorytm dopasowywania wag, mający na celu poprawną klasyfikację przykładów treningowych. Aktualizacja wag następuje tylko wtedy, gdy model popełni błąd.

### Parametry:

- **Learning rate**: $h > 0$ – współczynnik uczenia, decydujący o wielkości aktualizacji wag.
- **Epochs**: $N \in \mathbb{N}$ – liczba pełnych przejść przez zbiór treningowy.
- **Wektor wag początkowych**: $\mathbf{v} \in \mathbb{R}^d$ – inicjalizowany losowo lub jako wektor zerowy.

---

### Algorytm uczenia:

Dla każdego przykładu treningowego $(\mathbf{x}_i, y_i)$, gdzie:
- $\mathbf{x}_i \in \mathbb{R}^d$ – wektor cech,
- $y_i \in \{0, 1\}$ – etykieta klasowa,

wykonujemy:

1. **Obliczenie wejścia modelu**:

$$
z_i = \mathbf{v}^\top \mathbf{x}_i
$$

czyli iloczynu skalarnego wag i wektora cech.

2. **Predykcja modelu**:

$$
p_i = H(z_i)
$$

gdzie $H(z)$ to funkcja aktywacji Heaviside’a:

$$
H(z) =
\begin{cases}
1 & \text{jeśli } z \geq 0 \\\\
0 & \text{w przeciwnym razie}
\end{cases}
$$

3. **Sprawdzenie poprawności klasyfikacji**:
   - Jeśli $p_i = y_i$, to przykład został poprawnie sklasyfikowany – **brak zmian**.
   - Jeśli $p_i \ne y_i$, to dokonujemy aktualizacji wag:

$$
\mathbf{v} \leftarrow \mathbf{v} + h \cdot (y_i - p_i) \cdot \mathbf{x}_i
$$

**Intuicja aktualizacji:**
- Gdy $y_i = 1$, $p_i = 0$ → dodajemy $h \cdot \mathbf{x}_i$
- Gdy $y_i = 0$, $p_i = 1$ → odejmujemy $h \cdot \mathbf{x}_i$

**Uwaga:**  
Korekta wag zmienia wartość iloczynu skalarnego proporcjonalnie do normy przykładu:

$$
(\mathbf{v} \pm h \cdot \mathbf{x}_i)^\top \mathbf{x}_i = \mathbf{v}^\top \mathbf{x}_i \pm h \cdot \|\mathbf{x}_i\|^2
$$


In [ ]:
"""Funkcja Heaviside'a."""
def heaviside(x):
    return np.where(x >= 0, 1, 0)

"""Perceptron"""
class Perceptron():
    def __init__(self, learning_rate=0.1, epochs=20):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.weights = None

    def predict(self, X):
        return heaviside(np.dot(X, self.weights))

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.random.randn(n_features)

        for _ in range(self.epochs):
            for idx, x_i in enumerate(X):
                y_predicted = heaviside(np.dot(x_i, self.weights))

                update = self.learning_rate * (y[idx] - y_predicted)
                self.weights += update * x_i

    def accuracy(self, X, y):
        y_pred = self.predict(X)
        return np.mean(y_pred == y)


In [ ]:
def wizualizacja(X, y, perceptron):
    # Trenowanie perceptronu
    perceptron.fit(X, y)
    acc = perceptron.accuracy(X, y)

    # Wizualizacja wyników z kolejnymi barierami decyzyjnymi
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02), np.arange(y_min, y_max, 0.02))

    plt.figure(figsize=(6, 6))

    # Ostateczna bariera decyzyjna
    Z = perceptron.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    plt.contourf(xx, yy, Z, alpha=0.8, cmap=plt.cm.Paired)

    # Punkty danych
    plt.scatter(X[:, 0], X[:, 1], c=y, edgecolors='k', marker='o')
    plt.axhline(0, color='black', linewidth=1, linestyle='-')
    plt.axvline(0, color='black', linewidth=1, linestyle='-')
    plt.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)
    plt.title(f'Klasyfikacja perceptronem\nAccuracy: {acc:.2f}')
    plt.show()

perceptron_no_bias = Perceptron(learning_rate=0.1, epochs=100)

# Generowanie prostych danych
e=0.2
np.random.seed(42)
X = np.random.randn(500, 2)
y = np.array([1 if x[0] + x[1]> np.random.normal(loc=0, scale=e) else 0 for x in X])
wizualizacja(X,y, perceptron_no_bias)


# sytuacja gdy nie ma separowalności przez linię przechodzącą przez zero
y = np.array([1 if x[0] + x[1]+1> np.random.normal(loc=0, scale=e) else 0 for x in X])
wizualizacja(X,y, perceptron_no_bias)

## Rozszerzanie przestrzeni danych by umożliwić barierę postaci
$$
v^Tx+b \leq 0
$$

Rozszerzamy dane z $x$ do $(x,1)$, i na takich rozszerzonych stosujemy model.
W praktyce wagi modelu mają dodatkową współrzędną, nazywamy ją bias.

Wtedy pojawia się update zarówno dla $v$ jak i $b$:
$$
update=h\cdot (y_i -predicted(x_i))
$$
i
$$
weights=weights+h \cdot update \cdot x_i, \, bias=bias+h\cdot update.
$$

In [ ]:
class PerceptronBias():
    def __init__(self, learning_rate=0.1, epochs=20):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.weights = None
        self.bias = None

    def predict(self, X):
        return heaviside(np.dot(X, self.weights) + self.bias)

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.random.randn(n_features)
        self.bias = np.random.rand()

        for _ in range(self.epochs):
            for idx, x_i in enumerate(X):
                y_predicted = heaviside(np.dot(x_i, self.weights) + self.bias)

                update = self.learning_rate * (y[idx] - y_predicted)
                self.weights += update * x_i
                self.bias += update * 1

    def accuracy(self, X, y):
        y_pred = self.predict(X)
        return np.mean(y_pred == y)

perceptron_bias = PerceptronBias(learning_rate=0.1, epochs=100)

# Generowanie prostych danych
e=0.2
np.random.seed(0)

X = np.random.randn(500, 2)
y = np.array([1 if x[0] + x[1] +1> np.random.normal(loc=0, scale=e) else 0 for x in X])
wizualizacja(X,y, perceptron_bias)

Mówimy, że zbiory $X_0,X_1 \subset \mathbb{R}^n$ są separowalne liniowo, jeżeli istnieje $v \in \mathbb{R}^n$ i skalar $b$ taki, że
$$
v^TX_0+b <0, \, v^T X_1+b > 0
$$
Inaczej mówiąc dla każdego $x_i$ mamy
$$
H(v^T x_i+b)=y_i.
$$

Przykład zbioru nie separwoalnego liniowo (rozdzielamy O od X):
$$
\begin{array}{cc}
OX \\
XO
\end{array}
$$

*Twierdzenie* Jeżeli zbiory są separowalne liniowo, to algorytm uczenia perceptronu jest zbieżny do prawidłowego rozwiązania które rozdziela klasy.


# A co jeśli nie da się rozdzielić za pomocą liniowego modelu?


In [ ]:
perceptron = PerceptronBias(learning_rate=0.1, epochs=100)

# a co jeżeli koło?
y = np.array([1 if x[0]**2 + x[1]**2 > 1+np.random.normal(loc=0, scale=e) else 0 for x in X])
wizualizacja(X,y, perceptron)

# moons
from sklearn.datasets import make_moons
X, y = make_moons(n_samples=300, noise=0.2, random_state=42)
wizualizacja(X,y, perceptron)

# Zanurzanie w przestrzeń wysoko wymiarową
Rozszerzamy przestrzeń z wejściowej przestrzeni o więcej cech. Losowo wybrane - wybieramy losową macierz o dużym wymiarze i losowy bias, i obkładamy jakąś nieliniowością, na przykład funkcją $relu(x)=\max(0,x)$ albo $\tanh(x)$:

$$
x \to ReLU(Wx+b)
$$

In [ ]:
# F to zanurzenie w wiekszą przestrzeń -- reprezentacja

"""Perceptron z reprezentacją i augmentacją"""
class PerceptronRep:
    def __init__(self, learning_rate=0.1, epochs=400):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.weights = None
        self.bias=None

    def predict(self, x, F):
        return heaviside(np.dot(F(x), self.weights) + self.bias)

    def fit(self, X, y, F):
        n_features = len(F(X[0]))
        self.weights = np.random.randn(n_features)
        self.bias = np.random.rand()

        for _ in range(self.epochs):
            for idx, x_i in enumerate(X):
                noise=np.random.randn(len(x_i))
                y_predicted = heaviside(np.dot(F(x_i), self.weights) + self.bias)

                update = self.learning_rate * (y[idx] - y_predicted)
                self.weights += update * F(x_i)
                self.bias += update

    def accuracy(self, X, y, F):
        s=0
        S=0
        for idx, x_i in enumerate(X):
          s+=(heaviside(np.dot(F(x_i), self.weights) + self.bias) == y[idx])
          S+=1
        return s/S


def wizualizacjaRep(X_train,y_train,X_test,y_test, F):
  # Trenowanie perceptronu
  perceptron = PerceptronRep(learning_rate=1, epochs=100)
  perceptron.fit(X_train, y_train, F)
  acc_train = perceptron.accuracy(X_train, y_train, F)
  acc_test = perceptron.accuracy(X_test, y_test, F)

  # Wizualizacja wyników z kolejnymi barierami decyzyjnymi
  x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
  y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
  xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02), np.arange(y_min, y_max, 0.02))

  plt.figure(figsize=(6, 6))

  # Ostateczna bariera decyzyjna
  Z = perceptron.predict(np.c_[xx.ravel(), yy.ravel()],F)
  Z = Z.reshape(xx.shape)
  plt.contourf(xx, yy, Z, alpha=0.8, cmap=plt.cm.Paired)

  # Punkty danych
  plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, edgecolors='k', marker='x',alpha=0.7)
  plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, edgecolors='k', marker='o')
  plt.axhline(0, color='black', linewidth=1, linestyle='-')
  plt.axvline(0, color='black', linewidth=1, linestyle='-')
  plt.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)
  plt.title(f'Klasyfikacja perceptronem \nAccuracy train: {acc_train:.2f} test:{acc_test:.2f}')
  plt.show()


"""
Zanurzanie w przestrzeni wielowymiarowej
liniowe obłożone nieliniowościami
"""
dim=50
W = np.random.randn(2, dim)  # Macierz zanurzenia
b=np.random.randn(dim) # bias

# nieliniowość
def relu(x):
    return np.maximum(0, x)

# kluczowa funkcja zanurzająca w większą przestrzeń
def F(X):
  return relu(b+np.dot(X,W))

# Tworzenie zbioru danych Moon
np.random.seed(42)

N=100
N_train=N//2
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=N, noise=0.2, random_state=42)

X_train=X[:N_train]
X_test=X[N_train:]
y_train=y[:N_train]
y_test=y[N_train:]

np.random.seed(None)

wizualizacjaRep(X_train,y_train,X_test,y_test,F)


# 🧠 Zadanie – Rak Piersi (Breast Cancer Classification)

Twoim zadaniem jest klasyfikacja danych z zestawu **Breast Cancer** na podstawie dwóch cech:

- `mean radius`
- `mean texture`

Rozwiązanie wykonamy na dwa sposoby:

1. Implementacja własnego perceptronu  
2. Użycie gotowego modelu perceptronu z `sklearn.linear_model`

In [ ]:
from sklearn.datasets import load_breast_cancer

# Wczytanie danych
data = load_breast_cancer()
X = data.data[:, [0, 1]]  # tylko dwie cechy (mean radius, mean texture)

y = data.target

# Zamiana etykiet na -1 i 1
y = np.where(y == 0, -1, 1)

X_test=X[400:];
y_test=y[400:]
X=X[:400]
y=y[:400]

# Wizualizacja
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='bwr', edgecolor='k')
plt.xlabel(data.feature_names[0])
plt.ylabel(data.feature_names[1])
plt.title("Dane wejściowe: Breast Cancer")
plt.show()

# Wizualizacja granicy decyzyjnej
def plot_decision_boundary(X, y, model):
    # Zakres danych
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1

    # Siatka punktów
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02),
                         np.arange(y_min, y_max, 0.02))

    # Predykcja dla każdego punktu siatki
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    # Wykres
    plt.contourf(xx, yy, Z, alpha=0.4, cmap='bwr')
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap='bwr', edgecolor='k')
    plt.xlabel(data.feature_names[0])
    plt.ylabel(data.feature_names[1])
    plt.title("Perceptron - granica decyzyjna")
    plt.show()

In [ ]:
class Perceptron:
    def __init__(self, learning_rate=0.01, n_iters=1000):
        self.lr = learning_rate
        self.n_iters = n_iters
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        pass

    def predict(self, X):
        linear_output = np.dot(X, self.weights) + self.bias
        return np.sign(linear_output)

# Trenowanie perceptronu
perceptron = Perceptron(learning_rate=0.01, n_iters=1000)
perceptron.fit(X, y)



In [ ]:
# Wizualizacja
plot_decision_boundary(X_test, y_test, perceptron)

# Dokładność
y_pred = perceptron.predict(X)
accuracy = np.mean(y_pred == y)
print(f"Dokładność perceptronu: {accuracy:.2f}")

In [ ]:
# Twoja implementacja z użyciem perceptronu z SKLEARNt

## Klasyfikacja danych w kształcie kół za pomocą perceptronu

W tym zadaniu wykorzystamy **perceptron** do klasyfikacji danych, które układają się w **koncentryczne koła**. Celem jest sprawdzenie, jak klasyczny perceptron radzi sobie z danymi, które nie są liniowo separowalne.

### Kroki:
1. Wygeneruj dane w kształcie kół za pomocą `make_circles` z biblioteki `sklearn.datasets`.
2. Wyświetl dane na wykresie.
3. Zaimplementuj perceptron lub użyj gotowego z `sklearn.linear_model`.
4. Przeprowadź trening modelu na danych.
5. Oceń skuteczność modelu i zinterpretuj wyniki.


In [ ]:
from sklearn.datasets import make_circles

X,y=make_circles(random_state=42)